##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: ranking

This tutorial is a slightly adapted version of the [basic ranking tutorial](https://www.tensorflow.org/recommenders/examples/basic_ranking) from TensorFlow Recommenders documentation.



## Imports


Let's first get our imports out of the way.

In [8]:
!pip -q install tensorflow
!pip -q install tensorflow-recommenders
!pip -q install --upgrade tensorflow-datasets

In [9]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds



C:\Users\ricar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
# pip install tensorflow-recommenders
import tensorflow_recommenders as tfrs

In [10]:
# Cargar el dataset de tfds
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ricar\tensorflow_datasets\movielens\100k-ratings\0.1.1.incomplete0PGSXS\movielens-train.tfr…

Dataset movielens downloaded and prepared to C:\Users\ricar\tensorflow_datasets\movielens\100k-ratings\0.1.1. Subsequent calls will reuse this data.


Partiremos los datos para poner 80% de los ratings en el conjunto de entrenamiento, y 20% en el conjunto de test

In [11]:

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

A continuación, averiguamos los id de usuarios únicos y los títulos de películas presentes en los datos para que podamos crear las tablas de incrustación de usuarios y películas incrustadas.

In [12]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Implementacion del modelo

### Arquitectura

Ranking models do not face the same efficiency constraints as retrieval models do, and so we have a little bit more freedom in our choice of architectures. We can implement our ranking model as follows:

In [14]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    
  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

### Perdida y metricas de rendimiento

We'll make use of the `Ranking` task object: a convenience wrapper that bundles together the loss function and metric computation. 

We'll use it together with the `MeanSquaredError` Keras loss in order to predict the ratings.

In [19]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

### El modelo completo

We can now put it all together into a model.

In [20]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")
    
    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [21]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [22]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [23]:
model.fit(cached_train, epochs=3)

Epoch 1/3



10/10 [==============================] - 2s 27ms/step - root_mean_squared_error: 2.1834 - loss: 4.3796 - regularization_loss: 0.0000e+00 - total_loss: 4.3796
Epoch 2/3
10/10 [==============================] - 0s 15ms/step - root_mean_squared_error: 1.1205 - loss: 1.2575 - regularization_loss: 0.0000e+00 - total_loss: 1.2575
Epoch 3/3
10/10 [==============================] - 0s 16ms/step - root_mean_squared_error: 1.1105 - loss: 1.2353 - regularization_loss: 0.0000e+00 - total_loss: 1.2353


As the model trains, the loss is falling and the RMSE metric is improving.

Finally, we can evaluate our model on the test set:

In [24]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 14ms/step - root_mean_squared_error: 1.1042 - loss: 1.2148 - regularization_loss: 0.0000e+00 - total_loss: 1.2148


{'root_mean_squared_error': 1.1041843891143799,
 'loss': 1.1954413652420044,
 'regularization_loss': 0,
 'total_loss': 1.1954413652420044}

## Preparacion del dataset
Seguimos usando el conjunto de datos de MovieLens. Esta vez, también vamos a mantener las calificaciones: estos son los objetivos que estamos tratando de predecir.


Cuanto más baja sea la métrica RMSE, más preciso será nuestro modelo para predecir las calificaciones.

## Exporting for serving

The model can be easily exported for serving:


In [25]:
tf.saved_model.save(model, "exported-ranking/123")

INFO:tensorflow:Assets written to: exported-ranking/123\assets


INFO:tensorflow:Assets written to: exported-ranking/123\assets



We will deploy the model with TensorFlow Serving soon.

In [26]:
# Zip the SavedModel folder for easier download
!zip -r exported-ranking.zip exported-ranking/ 

"zip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
